# RGB channel image maker

Code that allows you to create manipulate image RGB channels and see how they affect your plot, on-the-fly.

In [ ]:
import EQ_tools as eq
from ipywidgets import interactive, VBox, Layout, widgets, fixed
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import os
import xarray as xr
import matplotlib

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.getcwd()) + '/Hub_Tools/')
import image_tools as img

In [ ]:
# image reduction settings: resample the image every grid_factor points
reduce_image = False
grid_factor = 5

# subset image: cut a relevant section out of an image. subset_extents [lon1,lon2,lat1,lat2] describes the section.
subset_image = True
subset_extents = [17.0, 22.0, 57.0, 60.0]

# image truncation settings
truncate_image = True
min_percentile = 5
max_percentile = 95

# image normalisation settings
channel_contrast = [1.0, 1.0, 1.0] # r,g,b
channel_brightness = 1.0

# image histogram settings
histogram_image = True
histogram_channels = 512

In [ ]:
vardir = '/home/jovyan/work/products/S3A_OL_1_EFR____20190809T092510_20190809T092810_20190810T145535_0179_048_036_1980_MAR_O_NT_002.SEN3'
# read lat/lon
ds1 = xr.open_dataset(os.path.join(vardir, 'geo_coordinates.nc'))
raster_lat = ds1.latitude.data
raster_lon = ds1.longitude.data
ds1.close()

# open data
num_channels = 11

if 'EFR' in unzipped_file:
    radiometry_type = 'Oa%s_radiance'
else:
    radiometry_type = 'Oa%s_reflectance'

for rad_channel_number in range(1, num_channels+1):
    rad_channel = radiometry_type % (str(rad_channel_number).zfill(2))
    rad_file = os.path.join(unzipped_file, rad_channel + '.nc') 
    rad_fid = xr.open_dataset(rad_file)
    exec("Ch%s = rad_fid.%s.data" % (str(rad_channel_number).zfill(2),rad_channel))
    rad_fid.close()

if 'EFR' in unzipped_file:
    red = np.log10(1.0 + 0.01 * Ch01 + 0.09 * Ch02 + 0.35 * Ch03 + 0.04 * Ch04 + 0.01 * Ch05 + 0.59 * Ch06 + 0.85 * Ch07 + 0.12 * Ch08 + 0.07 * Ch09 + 0.04 * Ch10)
    green = np.log10(1.0 + 0.26 * Ch03 + 0.21 * Ch04 + 0.50 * Ch05 + Ch06 + 0.38 * Ch07 + 0.04 * Ch08 + 0.03 * Ch09 + 0.02 * Ch10)
    blue = np.log10(1.0 + 0.07 * Ch01 + 0.28 * Ch02 + 1.77 * Ch03 + 0.47 * Ch04 + 0.16 * Ch05)

In [ ]:
%matplotlib inline

nchan = len(channel_red)
print('nChannels: '+str(nchan))

# build the widgets
red_widgets, green_widgets, blue_widgets = eq.make_widgets(channel_red, channel_green, channel_blue)

# build the plot command
runCMD = eq.build_plot_command(vlimits,\
                               red_widgets, green_widgets, blue_widgets)
# run the plot command
exec(runCMD)
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center')

# display the plot
for ii in range(0, nchan):
    display(VBox([iplot.children[ii], iplot.children[ii+nchan],iplot.children[ii+nchan*2]],\
                  layout=box_layout))
display(iplot.children[-1])

In [ ]:
# interpolate colour map onto required range / steps
I_am_done = False

if I_am_done:
    reds = interactive_plot.result[2]
    greens = interactive_plot.result[3]
    blues = interactive_plot.result[4]
    tscale_orig = np.linspace(vmin, vmax, len(reds))
    tscale_new = np.linspace(vmin, vmax, vsteps)

    interp_redf = interpolate.interp1d(tscale_orig, reds)
    interp_greenf = interpolate.interp1d(tscale_orig, greens)
    interp_bluef = interpolate.interp1d(tscale_orig, blues)

    interp_red = np.array(interp_redf(tscale_new)*255.).astype(int)
    interp_green = np.array(interp_greenf(tscale_new)*255.).astype(int)
    interp_blue = np.array(interp_bluef(tscale_new)*255.).astype(int)

In [ ]:
if I_am_done:
    # write to file
    my_file = 'Test_colbar.grass'
    if os.path.exists(my_file):
        os.remove(my_file)

    with open(my_file, 'w') as the_file:
        for ii in range(0,len(interp_red)):
            the_file.write(str(tscale_new[ii]) + ' ' + \
                           str(interp_red[ii]) + ' ' + \
                           str(interp_green[ii]) + ' ' + \
                           str(interp_blue[ii]))
            if ii != len(interp_red)-1:
                the_file.write('\n')